In [2]:
import os
import cv2
import shutil
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image

In [3]:
# 원본 데이터셋 경로
source_dataset_path  = '/content/drive/MyDrive/Python_project/Data/current_dataset'
resized_dataset_path = '/content/drive/MyDrive/Python_project/Data/resized_dataset'
model_path = '/content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/model'

# 쓰레기 카테고리
categories = ['glass','metal','paper','plastic','vinyl']
sizes = []

# 이미지 리사이즈 크기 결정
resize_width, resize_height = 528, 532

In [4]:
for category in categories:
    category_path = os.path.join(source_dataset_path, category)
    image_files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for filename in image_files[:]:
        file_path = os.path.join(category_path, filename)
        image = cv2.imread(file_path)
        if image is not None:
          sizes.append(image.shape[:2])

In [5]:
# 카테고리별로 폴더를 순회하며 이미지 처리
for category in categories:
    source_category_path = os.path.join(source_dataset_path, category)
    resized_category_path = os.path.join(resized_dataset_path, category)
    os.makedirs(resized_category_path, exist_ok=True)

    # 이미지 파일 처리
    image_files = os.listdir(source_category_path)[:]
    for filename in image_files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(source_category_path, filename)
            image = cv2.imread(file_path)
            if image is not None:
                resized_image = cv2.resize(image, (resize_width, resize_height))
                cv2.imwrite(os.path.join(resized_category_path, filename), resized_image)

In [13]:
def scale_pixels(image):
    """이미지 픽셀 값을 -1에서 1 사이의 값으로 스케일링합니다."""
    return (image / 127.5) - 1

train_datagen = ImageDataGenerator(
    preprocessing_function=scale_pixels,
    rotation_range=10,  # 회전 범위를 줄임
    width_shift_range=0.05,  # 이동 범위를 줄임
    height_shift_range=0.05,  # 이동 범위를 줄임
    shear_range=0.05,  # 전단 변환 범위를 줄임
    zoom_range=0.05,  # 확대/축소 범위를 줄임
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# 참고 : https://tykimos.github.io/2017/06/10/CNN_Data_Augmentation/

# 학습 데이터셋 로더 설정
train_generator = train_datagen.flow_from_directory(
    resized_dataset_path,
    target_size=(resize_width, resize_height),
    batch_size=16,
    class_mode='categorical',
    subset='training'  # 학습 데이터셋
)

# 검증 데이터셋 로더 설정
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 검증 데이터셋도 정규화 필요
validation_generator = validation_datagen.flow_from_directory(
    resized_dataset_path,
    target_size=(resize_width, resize_height),
    batch_size=16,
    class_mode='categorical',
    subset='validation'  # 검증 데이터셋
)

Found 6644 images belonging to 5 classes.
Found 1659 images belonging to 5 classes.


In [14]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(resize_width, resize_height, 3))

In [15]:
model = models.Sequential()
model.add(vgg_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(5, activation='softmax'))  # 클래스 수에 따라 조정

In [16]:
vgg_base.trainable = False

In [19]:
from tensorflow.keras.optimizers import Adam

# 모델 컴파일
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = '/content/drive/MyDrive/Python_project/Data/model_checkpoint.h5'

# 모델 체크포인트 콜백 생성
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

# 모델 학습 코드에 콜백을 추가
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint_callback]  # 콜백 리스트에 추가
)


Epoch 1/10
416/416 [==============================] - ETA: 0s - loss: 1.3422 - accuracy: 0.7816

In [ ]:
# 모델 평가
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")

# 테스트 이미지에 대한 예측 수행
for image_file in test_image_files:
    img_array = load_and_preprocess_image(image_file)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = categories[predicted_class_index]
    print(f"Image: {image_file}, Predicted class: {predicted_class_name}")